## NetworkX: Creating Graphs, Finding Paths, Exporting Data

This notebook demonstrates how to use they Python library NetworkX to find the shortest path in a graph data structure.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

### Create a very simple graph using an existing data structure in NetworkX

This graph has four nodes, labeled 1, 2, 3, and 4. In this example we'll ultimately want to find the shortest path between nodes 1 and 4. Based on the edge weights the correct path is 1, 2, 4. 

Note that NetworkX includes methods to draw graphs but they don't seem to work very well, probably because nothing in the graph specification indicates where to put the nodes.

<img src="simple_graph.jpg", width=25%, align=left>

In [2]:
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4])
# third element in each tuple is automatically labeled "weight".
elist=[(1,2,2.0),(1,3,1.0),(2,4,1.0),(3,4,5.0)]
G.add_weighted_edges_from(elist)

### Now that we have a graph, demonstrate some of the stuff we can do with it with NetworkX:

There's more good information here: https://networkx.readthedocs.org/en/latest/tutorial/tutorial.html#nodes. Review this to understand method on directed graphs!

In [3]:
# Get some basic info about the graph:
G.number_of_nodes(), G.number_of_edges()

(4, 4)

In [4]:
# Find all the nodes:
G.nodes()

[1, 2, 3, 4]

In [5]:
# Find all the nodes, but return an iterator.
# Returning a list (as above) probably isn't necessary if we're just going to iterate through it.
# Can do the same with "for n in G:"
G.nodes_iter()

<dictionary-keyiterator at 0x108a6ff70>

In [6]:
# Find all the edges:
G.edges()

[(1, 2), (1, 3), (2, 4), (3, 4)]

In [7]:
# Iterate through the neighbors of a given node (in this example, node 1):
for nbr in G[1]:
    print nbr

2
3


In [8]:
G.neighbors(1)

[2, 3]

In [9]:
# networkx.all_neighbors accomplishes the same thing:
for nbr in nx.all_neighbors(G, 1):
    print nbr

2
3


In [10]:
# Get the attributes associated with a particuar edge.
# If the specified edge doesn't exist, returns None or the specified default value.
G.get_edge_data(1,2)

{'weight': 2.0}

In [11]:
G.get_edge_data(1,4, default='No edge found!')

'No edge found!'

In [12]:
# If an edge exists you can add an attribute with subscript notation:
G[1][2]['color']='blue'
G.get_edge_data(1,2)

{'color': 'blue', 'weight': 2.0}

In [13]:
# Another way to set and change edge attributes:
nx.set_edge_attributes(G, 'another_attribute', 10)
nx.set_edge_attributes(G, 'color', 'red')

In [14]:
G.get_edge_data(1,2)

{'another_attribute': 10, 'color': 'red', 'weight': 2.0}

In [15]:
# Get the attributes associated with a particular node.
# In this example graph the nodes don't have any attributes.

In [16]:
G.node[1]

{}

In [17]:
# Another way to get all nodes with their attributes:
G.nodes(data=True)

[(1, {}), (2, {}), (3, {}), (4, {})]

### Now find the shortest path between two nodes:

In [18]:
# Find the path (sequence of nodes):
nx.dijkstra_path(G, 1, 4, weight='weight')

[1, 2, 4]

In [19]:
# Find the total path length:
nx.dijkstra_path_length(G, 1, 4)

3.0

In [20]:
# This method returns the path and length:
nx.single_source_dijkstra(G, 1, 4)

({1: 0, 2: 2.0, 3: 1.0, 4: 3.0}, {1: [1], 2: [1, 2], 3: [1, 3], 4: [1, 2, 4]})

### Experiment with ways of exporting the graph data

#### Option 1: dict of dicts. Single data structure contains all graph information.

In [21]:
nx.to_dict_of_dicts(G)

{1: {2: {'another_attribute': 10, 'color': 'red', 'weight': 2.0},
  3: {'another_attribute': 10, 'color': 'red', 'weight': 1.0}},
 2: {1: {'another_attribute': 10, 'color': 'red', 'weight': 2.0},
  4: {'another_attribute': 10, 'color': 'red', 'weight': 1.0}},
 3: {1: {'another_attribute': 10, 'color': 'red', 'weight': 1.0},
  4: {'another_attribute': 10, 'color': 'red', 'weight': 5.0}},
 4: {2: {'another_attribute': 10, 'color': 'red', 'weight': 1.0},
  3: {'another_attribute': 10, 'color': 'red', 'weight': 5.0}}}

#### Option 2: dict of lists and edge list. The second data structure contains all the necessary information for the graph, but maybe the first enables quicker lookups for connectedness given a node?

In [22]:
nx.to_dict_of_lists(G)

{1: [2, 3], 2: [1, 4], 3: [1, 4], 4: [2, 3]}

In [23]:
nx.to_edgelist(G)

[(1, 2, {'another_attribute': 10, 'color': 'red', 'weight': 2.0}),
 (1, 3, {'another_attribute': 10, 'color': 'red', 'weight': 1.0}),
 (2, 4, {'another_attribute': 10, 'color': 'red', 'weight': 1.0}),
 (3, 4, {'another_attribute': 10, 'color': 'red', 'weight': 5.0})]

#### Option 3: numpy adjacency matrix. The entries indicate edge weights.

In [24]:
nx.to_numpy_matrix(G)

matrix([[ 0.,  2.,  1.,  0.],
        [ 2.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  5.],
        [ 0.,  1.,  5.,  0.]])

### Export to CSV

In [25]:
def process_edgelist(edge):
    return edge[0], edge[1], edge[2]['weight']

In [26]:
edgelist = map(process_edgelist, nx.to_edgelist(G))
# This writes a CSV of floats, even thought nodes are integers. Not sure if this is an issue.
# This might not be an issue if I used Pandas to_csv.
np.savetxt('edgelist.csv.gz', edgelist)

In [14]:
from itertools import count
from heapq import heappush, heappop

### Experimentation with a self coded version of Dijkstra's algorithm

In [19]:
def dijkstra(G, source, weight="distance"):
    """ Calculate all shortest paths from a given source node using Dijkstra's algorithm.
    
    Args:
        G: a NetworkX graph object
        source: the starting node for the shortest path algorithm
        
    Returns: a tuple holding two dictionaries. The first indicates the length of the 
        shortest path to every node in G. The second indicates the previous node for
        any given node along the shortest path.
    
    """
    push = heappush
    pop = heappop
    
    dist = dict()
    prev = dict()
    q = []
    inf = float("inf")
    c = count()
    
    for v in G.nodes_iter():
        dist[v] = inf
        prev[v] = None
        push(q, (inf, next(c), v))
    
    push(q, (0, next(c), source))
    dist[source] = 0
    visited = set()
    
    while q:
        d, _, u = pop(q)
        if u in visited:
            continue
        print "visiting node", u
        visited.add(u)
        for v in G[u]:
            alt = dist[u] + G.get_edge_data(u, v)[weight]
            if alt < dist[v]:
                print "updating distance for node", v
                dist[v] = alt
                prev[v] = u
                push(q, (alt, next(c), v))
    return dist, prev  

In [18]:
dijkstra(G, 1, weight="weight")

visiting node 1
updating distance for node 2
updating distance for node 3
visiting node 3
updating distance for node 4
visiting node 2
updating distance for node 4
visiting node 4


({1: 0, 2: 2.0, 3: 1.0, 4: 3.0}, {1: None, 2: 1, 3: 1, 4: 2})